In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import os
import shutil
import regionmask
import rioxarray

from itertools import product
from zarr.errors import GroupNotFoundError

from shapely.geometry import Polygon

import carbonplan_trace.v1.utils as utils
from s3fs import S3FileSystem

fs = S3FileSystem()

https://daac.ornl.gov/ABOVE/guides/Annual_30m_AGB.html


In [ ]:
d = "s3://carbonplan-climatetrace/validation/ABoVE/"
files = [f for f in fs.ls(d) if not f.endswith("/") and "SE" not in f]
years = [2010, 2014]

## Download data


In [ ]:
# for year in years:
#     data_by_year = []
#     for f in files:
#         print(f)
#         ds = xr.open_rasterio(f"s3://{f}").assign_coords({'band': np.arange(1984, 2015)}).sel(band=year)
#         ds = ds.drop('band').assign_coords({'x': np.round(ds.x, 5), 'y': np.round(ds.y, 5)})
#         data_by_year.append(ds.load())
#     data_by_year = xr.combine_by_coords(data_by_year, combine_attrs='drop_conflicts')
#     data_by_year.to_dataset(name='biomass').to_zarr(f'temp_{year}.zarr', mode='w', consolidated=True)

## Regrid data to Hansen's grid


In [ ]:
# based on the map shown in https://daac.ornl.gov/ABOVE/guides/Annual_30m_AGB.html,
# identify all tiles of interest

tiles = [
    "60N_140W",
    "60N_130W",
    "60N_120W",
    "60N_110W",
    "70N_170W",
    "70N_160W",
    "70N_150W",
    "70N_140W",
    "70N_130W",
    "70N_120W",
]

# tiles = ['60N_130W']

In [ ]:
# then reproject into each tile and save
for year in years:
    ds = xr.open_zarr(f"temp_{year}.zarr")
    ds = ds.where(ds < 65535)
    for tile in tiles:
        print(tile)
        output_path = d + f"{tile}_{year}.zarr"
        if fs.exists(output_path):
            continue
        template_path = f"s3://carbonplan-climatetrace/intermediate/ecoregions_mask/{tile}.zarr"
        template = xr.open_zarr(fs.get_mapper(template_path))["ecoregion"].rename(
            {"lat": "y", "lon": "x"}
        )
        template = template.rio.write_crs("EPSG:4326")
        ds_projected = ds.rio.reproject_match(template)
        ds_projected = ds_projected.where(ds_projected < 65535)
        ds_projected["biomass"] = ds_projected.biomass * 0.01 * 2
        ds_projected.to_zarr(fs.get_mapper(output_path), mode="w", consolidated=True)

In [ ]:
ds_projected = xr.open_zarr(fs.get_mapper(d + f"{tile}_{year}.zarr"))
ds_projected.biomass[::100, ::100].plot()

## Getting performance metric


In [ ]:
import xarray as xr
from numcodecs.zlib import Zlib
import pandas as pd
import numpy as np
import dask
from s3fs import S3FileSystem
import geopandas
from carbonplan_trace.v1 import utils
import regionmask

fs = S3FileSystem()

dask.config.set({"array.slicing.split_large_chunks": False})
from carbonplan_trace.v0.core import compute_grid_area

In [ ]:
# this should be combined with the above section
above_tiles = [
    "60N_140W",
    "60N_130W",
    "60N_120W",
    "60N_110W",
    "70N_170W",
    "70N_160W",
    "70N_150W",
    "70N_140W",
    "70N_130W",
    "70N_120W",
]
# above_tiles = ['60N_130W']
total_area = 0
total_pct_change = 0

for tile in above_tiles:
    # open hansen and crop to landsat area
    # doesn't need land mask because emissions there should be 0
    hansen = xr.open_zarr(f"s3://carbonplan-climatetrace/v0.4/tiles/30m/{tile}_tot.zarr").sel(
        year=slice(2011, 2013)
    )["emissions"]
    disturbed = (hansen > 0).any(dim="year")
    if disturbed.lat[0] > disturbed.lat[-1]:
        disturbed = disturbed.reindex(lat=disturbed.lat[::-1])

    # open Jon's data, constrain to hansen emission > 0, and calculate % change of each pixel
    jon_2010 = (
        xr.open_zarr(f"s3://carbonplan-climatetrace/validation/ABoVE/{tile}_2010.zarr")
        .drop(["spatial_ref", "band"])
        .rename({"x": "lon", "y": "lat"})
    )
    jon_2014 = (
        xr.open_zarr(f"s3://carbonplan-climatetrace/validation/ABoVE/{tile}_2014.zarr")
        .drop(["spatial_ref", "band"])
        .rename({"x": "lon", "y": "lat"})
    )

    if jon_2010.lat[0] > jon_2010.lat[-1]:
        jon_2010 = jon_2010.reindex(lat=jon_2010.lat[::-1])
    if jon_2014.lat[0] > jon_2014.lat[-1]:
        jon_2014 = jon_2014.reindex(lat=jon_2014.lat[::-1])

    disturbed = disturbed.assign_coords({"lat": jon_2010.lat, "lon": jon_2010.lon})
    disturbed = disturbed.where(jon_2010.biomass > 0)
    jon_2010 = jon_2010.where(disturbed == True).biomass
    jon_2014 = jon_2014.where(disturbed == True).biomass

    pct_change = ((jon_2014 - jon_2010) / jon_2010 * 100).compute()
    da_area = compute_grid_area(pct_change)
    da_area = da_area.where(disturbed == True)
    biomass_area_summed = (jon_2010 * da_area).sum().values
    pct_change_summed = (jon_2010 * da_area * pct_change.clip(max=0, min=-100)).sum().values

    print(tile, np.round(pct_change_summed / biomass_area_summed, 4))
    total_area += biomass_area_summed
    total_pct_change += pct_change_summed

print()
print("total pct change ", total_pct_change)
print("total area ", total_area)
print(f"average pct change {np.round(total_pct_change / total_area, 4)}")